# Python & SQLite – Teil 2

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# Verbindung zur Datenbank herstellen
database_path = r'/Users/Admin/Documents/1_augustkurs/08_SQL/databases/wohnungen.db'
connection = sqlite3.connect(database_path)

In [ ]:
# Cursor erstellen
cursor = connection.cursor()

In [ ]:
# Verbindung testen
cursor.execute('SELECT * FROM hotels;')

In [ ]:
# Das Ergebnis von fetchall() ist eine Liste
result = cursor.fetchall()

In [ ]:
result

In [ ]:
# Python-Wiederholung:
# Wie greifen wir auf das dritte Element der Liste zu?


In [ ]:
# Python-Wiederholung:
# Ein einzelnes Element des Ergebnisses von fetchall()
# ist ein Tupel. Wie greifen wir auf das vierte Element
# des dritten Tupels zu?


In [ ]:
# Warum haben wir fetchall auf Variable result gelegt?
# ...

## SQL-Abfrage-Ergebnisse direkt in DataFrames speichern

Daten aus Datenbank auslesen und in DataFrame speichern
-> Für die Umwandlung nutzen wir `pd.read_sql`

In [ ]:
help(pd.read_sql)

In [ ]:
hotels_df = pd.read_sql('SELECT * FROM hotels;', connection)
hotels_df.head()

#### Das Problem mit den Leerzeichen

In Python sollte man Leerzeichen (außerhalb von Strings) am besten vermeiden.
Allerdings sollte man auch innerhalb von Datenbanken auf Leerzeichen
verzichten, weil sie uns spätestens in Python wieder Probleme bereiten.

In [ ]:
# Alle Infos der Spalte "Preis in Mio" selektieren:
df_preis = pd.read_sql('SELECT Preis in Mio FROM hotels;', connection)

In [ ]:
# Der vorherige Code verursacht einen Fehler, da die Bezeichnung
# "Preis in Mio" nicht als zusammengehöriger String identifiziert wird
# Wie lösen wir das?


In [ ]:
connection.close()

### 2. Einmalige SQL-Verbindung
#### 2.1 `with`-Statement 

 Das `with`-Statement haben wir bereits im Zusammenhang mit dem Öffnen, Lesen und Schreiben von Dateien kennengelernt. Praktisch daran war, dass es eine Datei geöffnet und anschließend automatisch geschlossen hat. Den gleichen Vorteil können wir mit sqlite3 mit Datenbanken leider nicht nutzen.

In [ ]:
with sqlite3.connect(database_path) as connection:
    df = pd.read_sql('SELECT * FROM hotels;', connection)

df.head()

In [ ]:
# Trotz abgeschlossenem with Block, kann trotzdem auf Verbindung
# zugegriffen werden
connection.cursor()\
          .execute('SELECT * FROM hotels;')\
          .fetchall()

In [ ]:
# Manuelles schließen
connection.close()

ABER ganz nutzlos ist with mit sqlite3 auch nicht:
Wir können auf connection.commit() verzichten durch 'with'

In [ ]:
with sqlite3.connect(database_path) as connection:
    connection.execute('''INSERT INTO hotels
                          VALUES (12000, 15.5, 222, 'Zutzenhausen', 12.2)''')

In [ ]:
with sqlite3.connect(database_path) as connection:
    df = pd.read_sql("SELECT * FROM hotels;", connection)

In [ ]:
df.tail()

In [ ]:
connection.close()

#### 2.2 `try` - Statement

In [ ]:
# Alternativ dazu können wir einen try-except-finally-
# Block definieren
try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM hotels")
    print(cursor.fetchall())
except:
    print('Irgendwas hat da nicht geklappt.')
finally:
    connection.close()

## Prepared statement in SQL
"Prepared statements" verbessern zum einen die Laufzeit von Queries, welche mehrmals mit unterschiedlichen Werten aufgerufen werden. Außerdem bieten sie einen wichtigen Schutz gegen Angriffe von außen durch sogenannte SQL-Injections (Einschleusung von SQL-Code). 

### 1. Question Mark Style
1.1 Insert mit Question Mark style. Hier stehen '?' stellvertretend für beliebige Werte

In [ ]:
connection = sqlite3.connect(database_path)
cursor = connection.cursor()

In [ ]:
# Tabelle erstellen:
connection.execute(
    '''CREATE TABLE IF NOT EXISTS programmiersprachen(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(50),
            first_appeared INT);'''
)

In [ ]:
# Fragezeichen können als Platzhalter dienen, um dann einen Befehl
# mehrmals mit unterschiedlichen Werten ausführen zu können

# Die einzutragenden Werte, jedes Tupel ist ein Eintrag:
data = [
    ("Ada Lovelace Machinealgorithm", 1843),
    ("Assembler", 1949),
    ("COBOL", 1959),
    ("SQL", 1972),
    ("C++", 1985),
    ("Python", 1991),
    ("Swift", 2014)
]

In [ ]:
# Mit executemany kann man einen Befehl wiederholt auf eine iterierbare Datenstruktur anwenden.
# In unserem Fall liegt eine Liste aus Tupeln vor und executemany führt das Insert so oft aus,
# wie es Tupel in der Liste gibt.

# Da die Tupel jeweils über zwei Werte verfügen, werden sie von VALUES(?, ?) abgebildet:
cursor.executemany('''
                  INSERT INTO programmiersprachen(name, first_appeared)
                  VALUES (?, ?)''', data)

In [ ]:
# Was ist eigentlich mit der ID-Spalte?
# ...

In [ ]:
help(cursor.executemany)

In [ ]:
connection.commit()

In [ ]:
cursor.execute('SELECT * FROM programmiersprachen;').fetchall()

1.2 Tabelle ausgeben lassen

In [ ]:
result = cursor.execute('SELECT * FROM programmiersprachen;').fetchall()
for row in result:
    print(row)

In [ ]:
result = cursor.execute('SELECT * FROM programmiersprachen;').fetchall()
for _, name, year in result:
    print('Name der Sprache:', name)
    print('Erscheinungsjahr:', year)
    print()

1.3 Select mit Question Mark style

In [ ]:
abfrage = 'SELECT * FROM programmiersprachen WHERE name = ?;'
# (Python,) steht für ein Tupel mit nur EINEM Element:
cursor.execute(abfrage, ('Python',))
cursor.fetchall()

In [ ]:
abfrage = 'SELECT * FROM programmiersprachen WHERE name = ? OR first_appeared = ?;'
# (Python,) steht für ein Tupel mit nur EINEM Element:
cursor.execute(abfrage, ('Python', 1843))
cursor.fetchall()

1.4 Verwendung in Python-Funktionen

In [ ]:
# Funktion erstellen, mit der wir neue Zeilen
# in Tabelle 'hotels' eintragen können:
def add_hotel(gewinn, preis, qm, stadt, qm_preis):
    query = '''INSERT INTO hotels
               VALUES(?, ?, ?, ?, ?);'''
    cursor.execute(query, (gewinn, preis, qm, stadt, qm_preis))
    connection.commit()
    return 'Hotel wurde hinzugefügt!'

In [ ]:
# Funktion aufrufen:
add_hotel(125641, 15, 300, "Bamberg", 3500)

In [ ]:
# Zu programmiersprachen mit with-Schreibweise hinzufügen.
# Funktion (durch with kein commit mehr nötig!):
def add_language(name, year):
    query = '''INSERT INTO programmiersprachen(name, first_appeared)
               VALUES(?, ?);'''
    with connection:
        connection.execute(query, (name, year))
    return 'Programmiersprache hinzugefügt!'

In [ ]:
add_language('Rust', 2006)

In [ ]:
# Eventuelle Übungsaufgabe:
# Eine Funktion schreiben, die zu areas neue Zeilen hinzufügt.

### 2. Named style
2.1 Insert mit named style

In [ ]:
# Mithilfe eines Dictionaries können Keys der Zuordnung der Werte dienen
# Dadurch müssen Werte nicht in richtiger Reihenfolge stehen:
name_sql = """
            INSERT INTO programmiersprachen(name, first_appeared)
            VALUES(:name, :first_appeared);"""

sprache = {'first_appeared': 1993, 'name': 'Brainfuck'}

connection.execute(name_sql, sprache)

In [ ]:
connection.commit()

In [ ]:
cursor.execute("SELECT * FROM programmiersprachen")
cursor.fetchall()

2.2 Tabelle mit where-Bedingung ausgeben (named style)

In [ ]:
cursor.execute("""
               SELECT *
               FROM programmiersprachen
               WHERE first_appeared > :year;""",
               {"year": 1990})

cursor.fetchall()

In [ ]:
# Schließen nicht vergessen!
connection.close()

### 3. Warum nicht einfach f-String? >>> SQL-Injections!
Lasst uns eine ganze Tabelle voller "geheimer Inhalte" klauen!

In [ ]:
connection = sqlite3.connect(database_path)

In [ ]:
with connection:
    connection.execute('''CREATE TABLE IF NOT EXISTS super_confident(
                               password VARCHAR,
                               secret_content VARCHAR);
                               ''')

In [ ]:
data = [('21412452d', 'Daten zum Konto in der Schweiz'), 
        ('Zdsam832197m', 'Das größte Geheimnis'),
        ('998321_dsHwoepw§', 'Die Weltformel')]

with connection:
    connection.executemany('''INSERT INTO super_confident
                              VALUES(?, ?)''', data)

In [ ]:
cursor = connection.cursor()
cursor.execute('SELECT * FROM super_confident;')
cursor.fetchall()

In [ ]:
# So sollten Nutzer auf ihre Geheimnisse zugreifen:
code = 'Zdsam832197m'
cursor.execute(f'''SELECT * FROM super_confident WHERE password = '{code}';''')

In [ ]:
cursor.fetchall()

In [ ]:
# Jetzt kommt der "Angriff":
injection = '1 OR 1=1'
cursor.execute(f'''SELECT * FROM super_confident
                   WHERE password = {injection}''')

In [ ]:
# Die Katastrophe: Wir haben nun die komplette Tabelle!
cursor.fetchall()

In [ ]:
# Was passiert, wenn wir '?' schreiben?
injection = '1 OR 1=1'
cursor.execute('''SELECT * FROM super_confident
                   WHERE password = ?''', (injection,))

In [ ]:
# So sieht es schon besser aus!
cursor.fetchall()

In [ ]:
# Übungsaufgabe: Schreibe weiter an app.py und database.py von gestern.
# Schreibe die Funktion insert_row so, dass der Nutzer Datum und Lerninhalt eintragen kann,
# aber die Gefahr von SQL-Einschleusung nicht besteht!
# Schreibe die Funktion view_entries aus.
# Falls die Zeit reicht: Verbaue die Funktionen in app.py

In [ ]:
# Berücksichtige den Walrus-Operator:
# https://towardsdatascience.com/the-walrus-operator-in-python-a315e4f84583